In [1]:
import xarray as xrs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cfgrib
import math
import os
import FWI

ds = xrs.open_dataset("2015_abril_maio.grib", engine='cfgrib')
print(ds)

/home/ori/.local/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(


<xarray.Dataset> Size: 629kB
Dimensions:    (time: 61, latitude: 23, longitude: 16)
Coordinates:
  * time       (time) datetime64[ns] 488B 2015-04-01T12:00:00 ... 2015-05-31T...
    surface    float64 8B ...
  * latitude   (latitude) float64 184B 42.2 41.95 41.7 41.45 ... 37.2 36.95 36.7
  * longitude  (longitude) float64 128B 350.2 350.4 350.7 ... 353.4 353.7 353.9
Data variables:
    fbupinx    (time, latitude, longitude) float32 90kB ...
    drtcode    (time, latitude, longitude) float32 90kB ...
    dufmcode   (time, latitude, longitude) float32 90kB ...
    ffmcode    (time, latitude, longitude) float32 90kB ...
    fdsrte     (time, latitude, longitude) float32 90kB ...
    fwinx      (time, latitude, longitude) float32 90kB ...
    infsinx    (time, latitude, longitude) float32 90kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Con

In [2]:
def DistanceTwoPoints(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def find_nearest(lat, lon, latitudes, longitudes):
    min_distance = None
    nearest_point = None

    for lat2, lon2 in zip(latitudes, longitudes):
        distance = DistanceTwoPoints(lat, lon, lat2, lon2)
        if min_distance is None or distance < min_distance:
            min_distance = distance
            nearest_point = (lat2, lon2)

    return min_distance, nearest_point

def find_nearest_index(lat, lon, latitudes, longitudes):
    distances = np.array([DistanceTwoPoints(lat, lon, lat2, lon2) for lat2, lon2 in zip(latitudes, longitudes)])
    sorted_distances_indices = np.argsort(distances)
    nearest_points = [(latitudes[i], longitudes[i]) for i in sorted_distances_indices[:4]]
    min_distances = distances[sorted_distances_indices[:4]]
    return min_distances, nearest_points

def convert_longitude360(lon):
    if lon < 0:
        return lon + 360
    return lon

def convert_longitude(lon):
    if lon > 180:
        return lon - 360
    return lon


def total_precipitation(initial_date, final_date, dmet):
    dmet['hourly.time'] = pd.to_datetime(dmet['hourly.time'])
    dmet_copy = dmet.copy()
    dmet_copy.set_index('hourly.time', inplace=True)
    selected_rows = dmet_copy.loc[initial_date:final_date]
    total_precipitation = selected_rows['hourly.precipitation'].sum()

    return total_precipitation

In [3]:
fwinx_at_time = ds['fwinx'].sel(time='2015-04-29T12:00:00')

print(fwinx_at_time)

latitudes = fwinx_at_time.latitude
longitudes = fwinx_at_time.longitude

# Print the latitudes
print(latitudes.values)
print(len(latitudes.values))
print(longitudes.values)
print(len(longitudes.values))

novas_lat = []
novas_lon = []
for lat in range(len(latitudes)):
    for lon in range(len(longitudes)):
        novas_lat.append(latitudes[lat].values)
        novas_lon.append(longitudes[lon].values)

<xarray.DataArray 'fwinx' (latitude: 23, longitude: 16)> Size: 1kB
[368 values with dtype=float32]
Coordinates:
    time       datetime64[ns] 8B 2015-04-29T12:00:00
    surface    float64 8B ...
  * latitude   (latitude) float64 184B 42.2 41.95 41.7 41.45 ... 37.2 36.95 36.7
  * longitude  (longitude) float64 128B 350.2 350.4 350.7 ... 353.4 353.7 353.9
Attributes: (12/28)
    GRIB_paramId:                             260540
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      368
    GRIB_typeOfLevel:                         surface
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    ...                                       ...
    GRIB_name:                                Forest fire weather index
    GRIB_shortName:                           fwinx
    GRIB_units:                               Numeric
    long_name:                                Forest fire weather index
    uni

In [5]:
estacoes = [f for f in os.listdir("estacoes") if f.endswith('.csv')]

for est in estacoes:
    df = pd.read_csv("estacoes/" + est)
    LATITUDE_SING = df.iloc[0]['latitude']
    LONGITUDE_SING = df.iloc[0]['longitude']
    min_distance, nearest_point = find_nearest(LATITUDE_SING, convert_longitude360(LONGITUDE_SING), novas_lat, novas_lon)
    print(min_distance, nearest_point)

    print("Latitude: ", nearest_point[0])
    print("Longitude: ", nearest_point[1])

    fwinx_value = ds['fwinx'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    drtcode_value = ds['drtcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    dufmcode_value = ds['dufmcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    ffmcode_value = ds['ffmcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])

    dmet_filtered = df[df['hourly.time'].str.contains('T12:00')]
    #dmet_filtered.to_csv(f'estacoes12/MD_{est}', index=False)
    
    start_index = 0 + 1
    LAT = nearest_point[0]
    FFMCPrev = ffmcode_value.values.item()
    DMCPrev = dufmcode_value.values.item()
    DCPrev = drtcode_value.values.item()
    
    mts = {
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8:31, 
        9:30, 
        10:31}


    fwix_values = [None] * len(dmet_filtered)
    ffmc_values = [None] * len(dmet_filtered)
    dmc_values = [None] * len(dmet_filtered)
    dc_values = [None] * len(dmet_filtered)
    isi_values = [None] * len(dmet_filtered)
    bui_values = [None] * len(dmet_filtered)


    for i in range(start_index, len(dmet_filtered)):
        MONTH = (dmet_filtered.iloc[i]['hourly.time']).split('-')[1]
        DAY = (dmet_filtered.iloc[i]['hourly.time']).split('-')[2].split('T')[0]

        print(MONTH, DAY)

        TEMP = dmet_filtered.iloc[i]['hourly.temperature_2m']
        RH = dmet_filtered.iloc[i]['hourly.relative_humidity_2m']
        WIND = dmet_filtered.iloc[i]['hourly.wind_speed_10m']


        if(int(DAY) == 1):
            LAST_DAY = mts[int(MONTH) - 1]
            LAST_MONTH = str(int(MONTH) - 1)
            
            if(int(LAST_MONTH) < 10):
                LAST_MONTH = '0' + str(int(LAST_MONTH))
                
            initial_date = f'2015-{LAST_MONTH}-{LAST_DAY}T12:00'
            final_date = f'2015-{MONTH}-{DAY}T12:00'

        elif((int(DAY) < 10) and (int(DAY) > 1)):
            id = '0' + str(int(DAY)-1)
            fd = '0' + str(int(DAY))

            initial_date = f'2015-{MONTH}-{id}T12:00'
            final_date = f'2015-{MONTH}-{fd}T12:00'
        else:
            id = str(int(DAY)-1)
            fd = str(int(DAY))

            initial_date = f'2015-{MONTH}-{id}T12:00'
            final_date = f'2015-{MONTH}-{fd}T12:00'

    
        RAIN = total_precipitation(initial_date, final_date, df)

        FFMCPrev = FWI.FFMC(TEMP,RH,WIND,RAIN,FFMCPrev)
        DMCPrev = FWI.DMC(TEMP,RH,RAIN,DMCPrev,LAT,int(MONTH))
        DCPrev = FWI.DC(TEMP,RAIN,DCPrev,LAT,int(MONTH))
        isi = FWI.ISI(WIND, FFMCPrev)
        bui = FWI.BUI(DMCPrev,DCPrev)
        fwix = FWI.FWI(isi, bui)

        print("FFMC: ", FFMCPrev)
        print("DMC: ", DMCPrev)
        print("DC: ", DCPrev)
        print("ISI: ", isi)
        print("BUI: ", bui)
        print("FWI: ", fwix)

        fwix_values[i] = fwix
        ffmc_values[i] = FFMCPrev
        dmc_values[i] = DMCPrev
        dc_values[i] = DCPrev
        isi_values[i] = isi
        bui_values[i] = bui


    dmet_filtered['fwix'] = fwix_values
    dmet_filtered['ffmc'] = ffmc_values
    dmet_filtered['dmc'] = dmc_values
    dmet_filtered['dc'] = dc_values
    dmet_filtered['isi'] = isi_values
    dmet_filtered['bui'] = bui_values


    dmet_filtered.to_csv(f'estacoes12X/fwi_{est}', index=False)

16179.920885674475 (array(38.45), array(350.95))
Latitude:  38.45
Longitude:  350.95
04 30
FFMC:  83.00468604989575
DMC:  12.5994163625
DC:  221.79537683105468
ISI:  1.976479725674276
BUI:  22.065214395477575
FWI:  3.3010223595112946
05 01
FFMC:  83.73626379748428
DMC:  14.3132780225
DC:  227.78137683105467
ISI:  2.882445281408991
BUI:  24.74003354536412
FWI:  5.3809050810390735
05 02
FFMC:  83.78355329944912
DMC:  15.8707596785
DC:  233.65937683105466
ISI:  2.6623930796367925
BUI:  27.133989459513217
FWI:  5.281949692630432
05 03
FFMC:  82.99021554413395
DMC:  16.836156100500002
DC:  239.33937683105466
ISI:  2.389176238789571
BUI:  28.636310587185175
FWI:  4.909048055510362
05 04
FFMC:  69.63564509322373
DMC:  15.421558869625489
DC:  245.14537683105468
ISI:  3.6621785446694535
BUI:  26.65162872675118
FWI:  7.078821054916982
05 05
FFMC:  80.39339297615426
DMC:  17.90942256962549
DC:  250.75337683105468
ISI:  2.1270792562582437
BUI:  30.39214150103266
FWI:  4.535397858968149
05 06
FFMC:

/tmp/ipykernel_19112/1860192279.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_19112/1860192279.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_19112/1860192279.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

FFMC:  33.068454535556526
DMC:  6.056401858370724
DC:  130.91284285804701
ISI:  0.029585950938834895
BUI:  10.857102340265268
FWI:  0.018668430675104066
05 05
FFMC:  45.42730835004971
DMC:  4.261090368713367
DC:  123.59966210864818
ISI:  0.18419501849779907
BUI:  7.845958459495033
FWI:  0.0978798900818818
05 06
FFMC:  66.39726059452671
DMC:  5.968633644713366
DC:  128.34366210864818
ISI:  0.7116253168846524
BUI:  10.69395851341027
FWI:  0.44529446267083905
05 07
FFMC:  85.36317780432627
DMC:  9.418471308713366
DC:  134.29366210864816
ISI:  6.351497911030417
BUI:  16.026891568868574
FWI:  8.672791611519862
05 08
FFMC:  85.41100443251237
DMC:  11.153657514713366
DC:  139.54166210864815
ISI:  4.584825229644742
BUI:  18.59211487583689
FWI:  7.036882781328762
05 09
FFMC:  86.47154385074728
DMC:  13.718921418713366
DC:  145.40166210864817
ISI:  3.363238199935986
BUI:  22.201062419510524
FWI:  5.843119744616465
05 10
FFMC:  88.92832925530958
DMC:  17.475200706713366
DC:  152.03566210864818
IS

/tmp/ipykernel_19112/1860192279.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_19112/1860192279.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_19112/1860192279.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

FFMC:  87.42730609416475
DMC:  25.294248466637733
DC:  58.168000000000006
ISI:  6.474160485949049
BUI:  25.25080569318303
FWI:  11.21113006789701
05 15
FFMC:  87.47581419501869
DMC:  27.51305431463773
DC:  63.470000000000006
ISI:  4.698358341885653
BUI:  27.470501813977283
FWI:  8.983450969566029
05 16
FFMC:  88.30063179947167
DMC:  30.78545069463773
DC:  69.744
ISI:  5.1818975904203475
BUI:  30.731878316734722
FWI:  10.394647594482079
05 17
FFMC:  88.6661773227496
DMC:  34.16157387863773
DC:  76.072
ISI:  5.685617773428146
BUI:  34.096774723358514
FWI:  11.876859241314321
05 18
FFMC:  89.38225729164259
DMC:  37.95155121463773
DC:  82.67
ISI:  5.668436813665729
BUI:  37.871739263363054
FWI:  12.552255764928585
05 19
FFMC:  89.43141082030958
DMC:  40.80798731463773
DC:  88.026
ISI:  8.246705630586105
BUI:  40.720194688376466
FWI:  17.40123145374888
05 20
FFMC:  89.48058057703106
DMC:  43.38272879463773
DC:  93.166
ISI:  8.43159421986592
BUI:  43.290044881252406
FWI:  18.2592837919186
05

...

/tmp/ipykernel_19112/1860192279.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_19112/1860192279.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_19112/1860192279.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

FFMC:  88.52222573616216
DMC:  34.980480504
DC:  331.49376708984374
ISI:  15.180917891936575
BUI:  55.357205040832994
FWI:  31.205869304290744
05 05
FFMC:  78.1349455995071
DMC:  34.2975685081697
DC:  337.0117670898437
ISI:  1.7010180044091612
BUI:  54.68257193489521
FWI:  5.481146623899551
05 06
FFMC:  86.13068287788698
DMC:  37.8021655001697
DC:  342.9617670898437
ISI:  2.6601305310882926
BUI:  59.27163475376095
FWI:  8.72725250157799
05 07
FFMC:  91.35981758820117
DMC:  42.7410356601697
DC:  349.9917670898437
ISI:  9.97206361481937
BUI:  65.48843339583878
FWI:  25.451577537559167
05 08
FFMC:  88.84918234069475
DMC:  45.3236751201697
DC:  356.1217670898437
ISI:  6.587323743029624
BUI:  68.76728987151607
FWI:  19.440914352463956
05 09
FFMC:  89.4550817028851
DMC:  49.2631875441697
DC:  362.9717670898437
ISI:  5.126920981108016
BUI:  73.56532593272563
FWI:  16.757797420053684
05 10
FFMC:  92.65212117715112
DMC:  55.095055976169704
DC:  370.7937670898437
ISI:  8.584113854659291
BUI:  80

/tmp/ipykernel_19112/1860192279.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_19112/1860192279.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_19112/1860192279.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

FFMC:  89.2680099130008
DMC:  20.4551004504547
DC:  73.68765546208378
ISI:  8.30284197235076
BUI:  24.150346241734734
FWI:  13.362087146263107
05 13
FFMC:  89.72646895140157
DMC:  24.2611370124547
DC:  80.15965546208378
ISI:  8.778394238445273
BUI:  27.622042244153768
FWI:  14.939937992436445
05 14
FFMC:  86.97601359887149
DMC:  25.9118148324547
DC:  85.33565546208378
ISI:  6.1632344421647245
BUI:  29.46006550673446
FWI:  11.703254730370652
05 15
FFMC:  87.02437270535222
DMC:  28.004779532454698
DC:  90.40365546208378
ISI:  7.328493507361653
BUI:  31.564690371591812
FWI:  13.926034762529127
05 16
FFMC:  88.22220979584559
DMC:  30.9365097084547
DC:  96.06565546208378
ISI:  9.146839272931821
BUI:  34.27701635020032
FWI:  17.202610601858634
05 17
FFMC:  88.92853191968783
DMC:  34.1904774684547
DC:  101.97965546208378
ISI:  7.042532910015014
BUI:  37.20057953314872
FWI:  14.728881428169524
05 18
FFMC:  90.43230236557486
DMC:  38.0752305644547
DC:  108.09165546208378
ISI:  6.457272027710267

/tmp/ipykernel_19112/1860192279.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_19112/1860192279.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_19112/1860192279.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta